In [1]:
import torch
from torchvision import transforms
import os
from PIL import Image
from matplotlib import pyplot as plt
from torch.autograd import Variable
import cv2
from face_detect.haarcascade import haarcascade_detect
import numpy as np

In [2]:
class_dict = {
    0: 'neutral',
    1: 'happiness',
    2: 'surprise',
    3: 'sadness',
    4: 'anger'
}

In [3]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# function to turn photos to tensor
loader = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [4]:
model = torch.load('Y:/Masters_Content/Intro to AI/Project _1/FER_PLUS/Codes/model_86.pth',map_location=torch.device('cpu'))

In [5]:
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [7]:
for img in os.listdir('Y:\Masters_Content/Intro to AI/Project _1/FER_PLUS/Test Images/'):
    
    frame = cv2.imread('Y:/Masters_Content/Intro to AI/Project _1/FER_PLUS/Test Images/' + img)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(frame)
    for (x,y,w,h) in faces:
        cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 2)

        # takes the region of interest of the face only in gray
        roi_gray = gray[y:y+h, x:x+h]
        roi_color = frame[y:y+h, x:x+h]

        # predict the mood
        image = loader(roi_color).float()
        image = Variable(image, requires_grad=True)
        image = image.unsqueeze(0)
        outputs = model(image)
        _, pred = torch.max(outputs, 1)
        scale = 1 # this value can be from 0 to 1 (0,1] to change the size of the text relative to the image

        cv2.putText(frame, class_dict[pred.numpy()[0]], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0))
        cv2.imwrite('Y:/Masters_Content/Intro to AI/Project _1/FER_PLUS/Predicted Images/' + img, frame)    